In [7]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data.tasks import *
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from experiments.models import *
from experiments.training import *
from experiments.SL_bound import *
from experiments.DA_bound import *
from util.kl import *
from util.misc import *
from results.plotting import *

# Hyper-parameters
batch_size = 128
num_classes = 10
epochs = 10
make_plots = False
delta=0.05 ## what would this be?   

TASK = 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and structure data

In [5]:
x_source,y_source,x_target,y_target=load_task(TASK)

mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.0025883]]


In [4]:
# ##### Add the label shifted datasets to each other creating the source and target domain for task 2

# ##### calculate the label densities here
# densities=[]
# densities.append(np.sum(y_shift,axis=0))
# densities.append(np.sum(y_shift_m,axis=0))
# densities.append(np.sum(y_shift_target,axis=0))
# densities.append(np.sum(y_shift_target_m,axis=0))
# # mnist source, mnist-m source, mnist target,  mnist-m target
# #print(densities)
# TASK=2
# if TASK==1:
#     ###### label density shifted mnist
#     x_source=x_shift
#     y_source=y_shift
#     x_target=x_shift_target
#     y_target=y_shift_target
# elif TASK==2:
#     #### MIXED MNIST and MNIST-m
#     L=len(densities[0])
#     interdomain_densities = [[] for x in range(2)]
#     for i in range(L):
#         ## all densities are # in mnist over # in mnist-m
#         interdomain_densities[0].append(densities[0][i]/densities[1][i])
#         interdomain_densities[1].append(densities[2][i]/densities[3][i])
#     print(interdomain_densities)
#     x_source=np.append(x_shift,x_shift_m, axis=0)
#     y_source=np.append(y_shift,y_shift_m, axis=0)
#     x_target=np.append(x_shift_target,x_shift_target_m, axis=0)
#     y_target=np.append(y_shift_target,y_shift_target_m, axis=0)
# elif TASK==3:
#     #### MNIST -> MNIST-m
#     x_source=x_full
#     y_source=y_full
#     x_target=x_full_m
#     y_target=y_full_m
# elif TASK==4:
#     #### MNIST->USPS
#     x_source=x_full
#     y_source=y_full
#     x_target=x_usps
#     y_target=y_usps
# elif TASK==5:
#     #### MNIST -> SVHN
#     x_source=x_full
#     y_source=y_full
#     x_target=x_svhn
#     y_target=y_svhn
# elif TASK==6:
#     x_source=x_chexpert
#     y_source=y_chexpert
#     x_source=x_chest14
#     y_source=y_chest14


[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.0025883]]


# Functions for evaluating bound

In [10]:


#### beta(T \| S), domain divergence, this is more or less equal to average label density ratio or worst case depending on parameter q

 #### lambda_\rho, difference in expected joint error |e_T-e_S| where e_S= E_h,h' E_x,y L(h(x),y)L(h'(x),y) 
    #### so draw two classifiers and calculate the mean of the product of losses on the domain   
def read_weights_germain(model,w_a,x_bound,y_bound,x_target,y_target,epochs_trained,sigma,epsilon,alpha,Binary=False,Task=TASK):
    import re
    KLs=[]
    e_s=[]
    e_t=[]
    d_tx=[]
    d_sx=[]
    epochs=[]
    train_germain=[] 
    target_germain=[]
    dis_rho=[]
    lambda_rho=[]
    sigma_tmp=sigma
    sigma=sigma[0]*10**(-1*sigma[1])
    
    ### Here we do something more intelligent to not have to hardcode the epoch amounts. 
    ### we parse the filenames and sort them in numerical order and then load the weights
    if Binary:
        path="posteriors/"+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))
    else:
        path="posteriors/"+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))
    list1=[]
    list2=[]
    dirFiles = os.listdir(path) #list of directory files
    ## remove the ckpt.index and sort so that we get the epochs that are in the directory
    for files in dirFiles: #filter out all non jpgs
        if '.ckpt.index' in files:
            name = re.sub('\.ckpt.index$', '', files)
            ### if it has a one it goes in one list and if it starts with a two it goes in the other
            if (name[0]=="1"):
                list1.append(name)
            elif (name[0]=="2"):
                list2.append(name)

    list1.sort(key=lambda f: int(re.sub('\D', '', f)))
    num_batchweights=len(list1)
    list2.sort(key=lambda f: int(re.sub('\D', '', f)))
    list1.extend(list2)
    Ws=list1 ## vector of checkpoint filenames

    Xvector=[]
    for i in Ws:
        #print(i)
        if i[0]=="1":
            if i[1]=="_":
                Xvector.append(int(i[2:]))
    for i in list2:
        Xvector.append((int(i[2:])+1)*547) ## TODO contant hack
    print(Xvector)
    """   
    epochs = [] #list of checkpoint filenames
    dirFiles = os.listdir(path) #list of directory files
    ## remove the ckpt.index and sort so that we get the epochs that are in the directory
    for files in dirFiles: #filter out all non jpgs
        if '.ckpt.index' in files:
            name = re.sub('\.ckpt.index$', '', files)
            epochs.append(name)
    epochs.sort(key=lambda f: int(re.sub('\D', '', f)))
    epochs=[int(i) for i in epochs]
    """ 
    path="posteriors/"+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"/"#"{epoch:0d}.ckpt"
    if Binary:
        path="posteriors/"+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"/"#"{epoch:0d}.ckpt"
    
    
    L=len(Xvector)
     ### vectors for saving std of each point
    error_std=[]
    target_error_std=[]
    e_s_std=[]
    e_t_std=[]
    d_tx_std=[]
    d_sx_std=[]
    for checkpoint in Ws:
        if Binary:
            model=init_task_model(TASK,True)
            model.compile(loss=tf.keras.losses.categorical_crossentropy,
                   optimizer=tf.keras.optimizers.SGD(learning_rate=0.003, momentum=0.95),
                      metrics=['accuracy'],)
        else:
            model=init_task_model(TASK,False)
            model.compile(loss=tf.keras.losses.categorical_crossentropy,
                   optimizer=tf.keras.optimizers.SGD(learning_rate=0.003, momentum=0.95),
                      metrics=['accuracy'],)

        model.load_weights(path+str(checkpoint)+".ckpt")
        w_s=model.get_weights()

        
        ###### for each pair of drawn prior and posterior we calculate the necessary parts of the bound 
        ###### and then average the result and return that
        
        n_classifiers=10
       

        e_s, e_t, d_sx, d_tx, e_s_std, e_t_std, d_sx_std, d_tx_std, train_germain, target_germain, error_std, target_error_std=draw_classifier_and_calculate_errors(w_s,sigma,n_classifiers,x_bound,y_bound,x_target,y_target,model)
        
        KLsum=0
        t = time.time()
        
        KLsum=estimate_KL(w_a,w_s,sigma)## compute the KL
        ## only w_s, w_a here
                
      
        KLs.append(KLsum)
        elapsed = time.time() - t
        print("Time spent calculating KL: "+str(elapsed)+"\n") 
        
        ### memory leak city
        del model
        _=gc.collect()
        
        
     
    print("Finished calculation of bound parts")
   
          #### load the result file if it exists otherwise make one
    if Binary:
        result_path="results/"+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
    else:
        result_path="results/"+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
    if(os.path.exists(result_path+"_results.pkl")):
        results=pd.read_pickle(result_path+"_results.pkl")
    else:
        results=pd.DataFrame({'Weightupdates': Xvector,
        'train_germain': train_germain,
        'target_germain':target_germain,
        'KL': KLs})
        with open(path_to_root_file+'mnist_transfer/'+result_path+"_results.pkl",'wb') as f:
            pickle.dump(results,f)
        f.close()
        results=pd.read_pickle(result_path+"_results.pkl")

    train_germain=np.array(train_germain)
    results['Weightupdates']=Xvector
    results['train_germain']=train_germain
    results['target_germain']=target_germain
    results['e_s']=e_s
    results['e_t']=e_t
    results['d_tx']=d_tx
    results['d_sx']=d_sx
    results['KL']=KLs
    ### save the std deviations as well in some form like a vector of deviations for each factor
    results['error_std']=error_std
    results['target_error_std']=target_error_std
    results['e_s_std']=e_s_std
    results['e_t_std']=e_t_std
    results['d_tx_std']=d_tx_std
    results['d_sx_std']=d_sx_std
    KL=KLs
    
    m=len(y_bound)
    delta=0.05 ## hardcoded value
     # calculate disrho bound
    [res,bestparam, boundparts]=grid_search(train_germain,e_s,e_t,d_tx,d_sx,KL,delta,m,L)
    # calculate beta bound
    [res2,bestparam2, boundparts2]=grid_search(train_germain,e_s,e_t,d_tx,d_sx,KL,delta,m,L,beta_bound=True)            
                
                
                
    results['germain_bound']=res
    print("Germain bound"+str(res))
    print("[a, omega]= "+str(bestparam))
    Best=np.zeros([len(res),1])
    Best[0]=bestparam[0]
    Best[1]=bestparam[1]
    Best[2]=CLASSIFIERS
    #print(Best)
    results['bestparam']=Best
    results['boundpart1_germain']=boundparts[0]
    results['boundpart2_germain']=boundparts[1]
    results['boundpart3_germain']=boundparts[2]
    results['boundpart4_germain']=boundparts[3]
    results['boundpart5_germain']=boundparts[4]
    ## beta bound
    results['beta_bound']=res2
    results['beta_boundpart1']=boundparts2[0]
    results['beta_boundpart2']=boundparts2[1]
    results['beta_boundpart3']=boundparts2[2]
    with open(path_to_root_file+'mnist_transfer/'+result_path+"_results.pkl",'wb') as f:
        pickle.dump(results,f)
    f.close()
    return results
def error_from_prediction(pred,y):
    ### note that this is for binary classification, when multiclass is used you have to change the 2 in the denominator to num_classes
    pred=make_01(pred)
    length=len(pred)
    return np.sum(np.abs(pred-y))/(2*length)
def grid_search(train_germain,e_s,e_t,d_tx,d_sx,KL,delta,m,L,beta_bound=False):
    #### here we want to do a coarse grid search over a and omega to get the smallest bound 
    print("Starting gridsearch....")
    avec=[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50,100,500,1000,5000,10000,50000,100000]
    omegas=[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50,100,500,1000,5000,10000,50000,100000]
    tmp= sys.maxsize
    res=[]
    bestparam=[0,0]
    for a in avec:
        for omega in omegas:
            if beta_bound:
                germain_bound, boundparts=calculate_beta_bound(e_s,d_tx,KL,delta,a,omega,m,L)
            else:
                germain_bound,boundparts=calculate_germain_bound(train_germain,e_s,e_t,d_tx,d_sx,KL,delta,a,omega,m,L)
            if min(germain_bound)<tmp:
                tmp=min(germain_bound)
                #print("Best bound thus far:"+str(tmp))
                res=germain_bound
                bestparam=[a,omega]
                
    ### do a finer sweep around the best parameters
    if bestparam[0]!=0:
        avec=np.arange(bestparam[0]-bestparam[0]/2,bestparam[0]+bestparam[0]*4,0.1*bestparam[0])
    else:## no bound better than the max int was found, if that is even possible
        avec=np.arange(-1,1,0.1)
    if bestparam[1]!=0:
        omegas=np.arange(bestparam[1]-bestparam[1]/2,bestparam[1]+bestparam[1]*4,0.1*bestparam[1])
    else:## no bound better than the max int was found
        avec=np.arange(-1,1,0.1)
    boundparts=[0, 0,0,0,0]
    for a in avec:
        for omega in omegas:
            if beta_bound:
                germain_bound, boundparts=calculate_beta_bound(e_s,d_tx,KL,delta,a,omega,m,L)
            else:
                germain_bound,boundparts=calculate_germain_bound(train_germain,e_s,e_t,d_tx,d_sx,KL,delta,a,omega,m,L)
            if min(germain_bound)<tmp:
                tmp=min(germain_bound)
                #print("Best finer bound thus far:"+str(tmp))
                res=germain_bound
                bestparam=[a,omega]
                #boundparts=[a1,a2,a3,a4,a5]
    return [res,bestparam, boundparts]
def calculate_beta_bound(e_s,d_tx,KL,delta,b,c,m,L,BETA=0):
    BETA=10.986111 ### hardcoded value for beta_infinity for TASK2 TODO!
    m_s=m  ## temporary, we should pass these in
    m_t=m  ## temporary, we should pass these in
    bprime=b/(1-np.exp(-b))
    cprime=c/(1-np.exp(-c))
    
    bound=[]
    a1=np.zeros(L)
    a2=np.zeros(L)
    a3=np.zeros(L)
    for i in range(L):
        a1[i]=cprime/2*(d_tx[i])
        a2[i]=bprime*e_s[i]
        a3[i]=(cprime/(m_t*c)+bprime*BETA/(m_s*b))*(2*KL[i]+np.log(2/delta))
    ## we cannot evaluate the eta term in the bound so this is it. For TASK 2 it is 0 anyway.
    ## And for other tasks we will not evaluate this bound anyway as we probably have no way of doing so easily..
        bound.append(a1[i]+a2[i]+a3[i])
    boundparts=[a1,a2,a3]
    return bound, boundparts

def germain_bound(x_bound,y_bound,x_target,y_target,alpha,sigma,epsilon,task,Binary=False):
    TASK=task
    sigma_tmp=sigma
    sigma=sigma[0]*10**(-1*sigma[1])
        ## read params.txt for the desired alpha and get the parameters
 
    if Binary:
        with open('posteriors/'+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+'/params.txt', 'rb+') as f:
            params=f.readlines()
        f.close()
        prior_path="priors/"+"task"+str(TASK)+"/Binary/"+str(int(100*alpha))+"/prior.ckpt"
        result_path="results/"+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"
    else:
        with open('posteriors/'+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+'/params.txt', 'rb+') as f:
            params=f.readlines()
        f.close()
        prior_path="priors/"+"task"+str(TASK)+"/"+str(int(100*alpha))+"/prior.ckpt"
        result_path="results/"+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"

    epsilon=float(params[1])
    epochs_trained=int(params[2])
    if Binary:
        M=init_task_model(TASK,True)
    else:
        M=init_task_model(TASK,False)
                
    M.compile(loss=tf.keras.losses.categorical_crossentropy,
                   optimizer=tf.keras.optimizers.SGD(learning_rate=0.003, momentum=0.95),
                      metrics=['accuracy'],)
      ### load the prior weights if there are any
    if(Binary and alpha != 0):
        prior_path="priors/"+"task"+str(TASK)+"/Binary/"+str(int(100*alpha))+"/prior.ckpt"
    elif(alpha != 0):
        prior_path="priors/"+"task"+str(TASK)+"/"+str(int(100*alpha))+"/prior.ckpt"
    if alpha==0:
        ### do nothing, just take the random initialisation
        w_a=M.get_weights()
    else:
        M.load_weights(prior_path)
        w_a=M.get_weights()
    
 
    ## get the prior weights for the KL and pass into read_weights
    results=read_weights_germain(M,w_a,x_bound,y_bound,x_target,y_target,epochs_trained,sigma_tmp,epsilon,alpha,Binary=Binary,Task=TASK)
    return results

In [11]:
def draw_classifier_and_calculate_errors(w_s,sigma,n_classifiers,x_bound,y_bound,x_target,y_target,M_posterior):
    errorsum=[]
    target_errorsum=[]
    e_ssum=[]
    e_tsum=[]
    d_txsum=[]
    d_sxsum=[]
    d_tx_h=0
    d_sx_h=0
    d_tx_hprime=0
    d_sx_hprime=0
    for i in range(n_classifiers):
        
        ### do a loop and draw two classifiers for each loop
        print('Drawing classifiers...')
        h = draw_classifier(w_s, sigma, num_classifiers=1)[0]
        hprime = draw_classifier(w_s, sigma, num_classifiers=1)[0]

        """
        Calculate train and target errors
        """
        y_bound = np.array(y_bound)
        y_target = np.array(y_target)

        ######## in here we should make the results save in a vector for each part to be able to calculate
        ######## the standard deviation and be able to get error bars on things.
        print('Calculating errors, joint errors and disagreements...')
        t = time.time()
        ### for the first draw
        M_posterior.set_weights(h)
        d_tx_h=M_posterior.predict(x_target,verbose=0)
        d_sx_h=M_posterior.predict(x_bound,verbose=0)
        d_sx_h=make_01(d_sx_h)
        d_tx_h=make_01(d_tx_h)

        errorsum.append(error_from_prediction(d_sx_h,y_bound))
        target_errorsum.append(error_from_prediction(d_tx_h,y_target))
        ### for the second draw
        M_posterior.set_weights(hprime)
        d_tx_hprime=M_posterior.predict(x_target,verbose=0)
        d_sx_hprime=M_posterior.predict(x_bound,verbose=0)
        d_sx_hprime=make_01(d_sx_hprime)
        d_tx_hprime=make_01(d_tx_hprime)
        errorsum.append(error_from_prediction(d_sx_hprime,y_bound))
        target_errorsum.append(error_from_prediction(d_tx_hprime,y_target))

        e_ssum.append(joint_error(d_sx_h,d_sx_hprime,y_bound))
        d_sxsum.append(classifier_disagreement(d_sx_h,d_sx_hprime))
        e_tsum.append(joint_error(d_tx_h,d_tx_hprime,y_target))
        d_txsum.append(classifier_disagreement(d_tx_h,d_tx_hprime))
        elapsed = time.time() - t
        print('Time spent calculating errors, joint errors and disagreements: %.4fs' % elapsed)

      

        

    train_germain = np.mean(errorsum) 
    target_germain = np.mean(target_errorsum)  
    error_std = np.std(errorsum)
    target_error_std = np.std(target_errorsum)
    # Means
    e_s = np.mean(e_ssum)
    d_sx = np.mean(d_sxsum)
    e_t = np.mean(e_tsum)
    d_tx = np.mean(d_txsum)

    # Std-devs
    e_s_std = np.std(e_ssum)
    d_sx_std = np.std(d_sxsum)
    e_t_std = np.std(e_tsum)
    d_tx_std = np.std(d_txsum)

    return e_s, e_t, d_sx, d_tx, e_s_std, e_t_std, d_sx_std, d_tx_std, train_germain, target_germain, error_std, target_error_std


# Run evaluation 

In [12]:
delta=0.05
Binary=True
epsilons=[0.1]
alphas=[0,0.1]#0,0.3]
sigmas=[[3,2]]

y_target_bin=make_mnist_binary(y_target)
y_source_bin=make_mnist_binary(y_source)

for alpha in alphas:
    '''
    if alpha==0:
        x_bound=x_source
        y_bound=y_source
        if Binary:
            y_bound=make_mnist_binary(y_bound)
            x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source_bin,test_size=alpha,random_state=69105)
        else:
            y_source_bin=make_mnist_binary(y_source)
            x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source_bin,test_size=alpha,random_state=69105)
    else:
        x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source,test_size=alpha,random_state=69105)
    '''
    print("alpha:"+str(alpha))
    if alpha==0:
        x_bound=x_source
        y_bound=y_source_bin
    else:
        x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source_bin,test_size=alpha,random_state=69105)
    for epsilon in epsilons:
        print("epsilon:"+str(epsilon))
        for sigma in sigmas:    
            print("sigma:"+str(sigma))
            results=germain_bound(x_bound,y_bound,x_target,y_target_bin,alpha,sigma,epsilon,TASK,Binary=Binary)
               

alpha:0
epsilon:0.1
sigma:[3, 2]


FileNotFoundError: [Errno 2] No such file or directory: 'posteriors/task2/Binary/100_0/params.txt'

# Plot results

In [ ]:
def plot_germain_res(alpha,epsilon,sigma,Binary=False,do_errorbars=True,plot_parts=False):
    sigma_tmp=sigma
    sigma=sigma[0]*10**(-1*sigma[1])
    import pandas as pd
    if Binary:
        result_path="results/"+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
        plt.title("Binary: "+r"$\alpha$="+str(alpha)+r" $\epsilon$="+str(epsilon)+r" $\sigma$="+str(sigma))
    else:
        result_path="results/"+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
        plt.title(r"$\alpha$="+str(alpha)+r" $\epsilon$="+str(epsilon)+r" $\sigma$="+str(sigma))
    results=pd.read_pickle(result_path+"_results.pkl") #
    num_classifiers=4#results['bestparam']
    #print(num_classifiers)
    
    N=num_classifiers*2 # how many values we got the estimate of the standard deviation from
    N2=num_classifiers
    z_alpha=1.96 ## in actuality we could probably use some degree of freedom thing here to make it more correct as
                 ## the standard deviation is unknown, but we do this for now and pretend like we know the standard deviation
        
    if do_errorbars==False:
        plt.plot(results["Weightupdates"],results["train_germain"],'k^-')
        plt.plot(results["Weightupdates"],results["target_germain"],'m^-')
    else:
        plt.errorbar(results["Weightupdates"],results["train_germain"],yerr=z_alpha*results['error_std']/N,fmt='k^-')
        plt.errorbar(results["Weightupdates"],results["target_germain"],yerr=z_alpha*results['target_error_std']/N,fmt='m^-')
    
    plt.xlabel("Weight updates")
    plt.ylabel("Error")
    if plot_parts==True:
        if do_errorbars==False:
            plt.plot(results["Weightupdates"],results['boundpart1_germain'],'*')
            plt.plot(results["Weightupdates"],results['boundpart2_germain'],'+-')
            plt.plot(results["Weightupdates"],results['boundpart3_germain'],'X')
            plt.plot(results["Weightupdates"],results['boundpart4_germain'],'D')
            plt.plot(results["Weightupdates"],results['boundpart5_germain'],'o-')
            plt.plot(results["Weightupdates"],results["germain_bound"],'r*-')
            plt.plot(results["Weightupdates"],results["beta_bound"])
            plt.legend(["Sample error","Target error","sample error part","dis_rho part","KL part","lambda_rho","extra constant","Dis-rho Bound","beta bound"], loc = 0)#,"e_s","e_t","d_tx","d_sx"])#,"Bound"
            plt.title(r"$\alpha$="+str(alpha)+r" $\epsilon$="+str(epsilon)+r" $\sigma$="+str(sigma))
            plt.show()
        else:
            #### 
            #### Errors here are calculated with the amount of classifiers drawn as the n that divides
            #### with n=2*(drawn classifiers) for errors and n=(drawn classifiers)^2 for other boundparts
            #### 
            ####
            plt.errorbar(results["Weightupdates"],results['boundpart1_germain'],yerr=z_alpha*results['error_std']/N,fmt='*')
            plt.errorbar(results["Weightupdates"],results['boundpart2_germain'],yerr=z_alpha*(results['d_tx_std']+results['d_sx_std'])/N2,fmt='+-')
            #plt.errorbar(results["Weightupdates"],results['boundpart3_germain'],yerr=z_alpha*results['_std']/N,fmt='X')
            plt.plot(results["Weightupdates"],results['boundpart3_germain'],'X')
            plt.errorbar(results["Weightupdates"],results['boundpart4_germain'],yerr=z_alpha*(results['e_t_std']+results['e_s_std'])/N2,fmt='D')
            plt.plot(results["Weightupdates"],results['boundpart5_germain'],'o-')
            #plt.errorbar(results["Weightupdates"],results['boundpart5_germain'],yerr=z_alpha*results['_std']/N,fmt='o-')
            plt.errorbar(results["Weightupdates"],results["germain_bound"],
                         yerr=z_alpha*(results['error_std']/N+(results['e_t_std']+results['e_s_std']+results['d_tx_std']+results['d_sx_std'])/N2),fmt='r*-')
            plt.errorbar(results["Weightupdates"],results["beta_bound"],
                         yerr=(results['e_s_std']+results['d_tx_std'])/N2)
            plt.legend(["Sample error","Target error","sample error part","dis_rho part","KL part","lambda_rho","extra constant","Dis-rho Bound","beta bound"], loc = 0)#,"e_s","e_t","d_tx","d_sx"])#,"Bound"
            plt.title(r"$\alpha$="+str(alpha)+r" $\epsilon$="+str(epsilon)+r" $\sigma$="+str(sigma))
            plt.show()
        
    else:
        if do_errorbars==False:
            plt.plot(results["Weightupdates"],results["germain_bound"],'r*-')
            plt.plot(results["Weightupdates"],results["beta_bound"])
            plt.legend(["Sample error","Target error","Dis-rho Bound","beta bound"], loc = 0)#,"e_s","e_t","d_tx","d_sx"])#,"Bound"
            plt.title(r"$\alpha$="+str(alpha)+r" $\epsilon$="+str(epsilon)+r" $\sigma$="+str(sigma))
            plt.show()
        else:
            disrho_error=(results['error_std']/N+(results['e_t_std']+results['e_s_std']+results['d_tx_std']+results['d_sx_std'])/N2)
            print(disrho_error)
            plt.errorbar(results["Weightupdates"],results["germain_bound"],
                         yerr=z_alpha*disrho_error,fmt='r*-',capsize=8)
            plt.errorbar(results["Weightupdates"],results["beta_bound"],
                         yerr=(results['e_s_std']+results['d_tx_std'])/N2,capsize=8)
            plt.legend(["Sample error","Target error","Dis-rho Bound","beta bound"], loc = 0)#,"e_s","e_t","d_tx","d_sx"])#,"Bound"
            plt.title(r"$\alpha$="+str(alpha)+r" $\epsilon$="+str(epsilon)+r" $\sigma$="+str(sigma))
            plt.savefig("Plot"+str(10*alpha)+".png",dpi=300)
            plt.show()
            #plt.savefig("Plot"+str(10*alpha)+".png",dpi=300)
sigma=[3,2]
#plot_germain_res(0,0.03,sigma,True)
plot_germain_res(0,0.03,sigma,True,True)
plot_germain_res(0.3,0.03,sigma,True,True)
#plot_germain_res(0.3,0.001,sigma,True)
#plot_germain_res(0.3,0.01,sigma,True)
#plot_germain_res(0.1,0.001,sigma,True)
#plot_germain_res(0.1,0.01,sigma,True)
#plot_germain_res(0.3,0.03,sigma,True)
#plot_germain_res(0.1,0.03,sigma,True)
#plot_germain_res(0.3,0.03,sigma,True)
#plot_germain_res(0.3,0.01,sigma,True)